In [1]:
import pandas as pd
from fullcontact import FullContactClient
import numpy as np
from imblearn.over_sampling import SMOTENC
from helpers import Data
from urllib.request import urlopen, Request
import mysql.connector
from getpass import getpass
from mysql.connector import connect
from bs4 import BeautifulSoup
import requests
import re
from helpers import individualFeatures, Data, Scraping

df = pd.read_csv(r'C:\Users\aacjp\Sales-Scientist\datasets\MockHyros.csv')
df1 = pd.read_csv(r'C:\Users\aacjp\Sales-Scientist\datasets\HyrosNumerical.csv')
df2 = pd.read_csv(r'C:\Users\aacjp\Sales-Scientist\datasets\HyrosAugmented.csv')

In [2]:
connection = connect(host='localhost', user='root', password='Raptor//Kona9', database='leads')
cursor = connection.cursor()

## tables to make
1. basic data entry
2. similarweb
3. twitter
4. zoominfo
5. landing page

The Hack: http://sangaline.com/post/advanced-web-scraping-tutorial/

In [3]:
def db2df():
    connection = connect(host='localhost', user='root', password='Raptor//Kona9', database='leads')
    cursor = connection.cursor()
    survey = pd.read_sql('''SELECT * FROM survey;''', connection)
    social = pd.read_sql('''SELECT * FROM social;''', connection).drop(['name'], axis='columns')
    landing = pd.read_sql('''SELECT * FROM landingpage;''', connection).drop(['name'], axis='columns')
    web = pd.read_sql('''SELECT * FROM similarweb;''', connection).drop(['id'], axis='columns')
    rank = pd.read_sql('''SELECT * FROM fullcontact;''', connection).drop(['id', 'name'], axis='columns')
    return pd.concat([survey, social, landing, web, rank], axis='columns')

df = db2df()
df

,name,landingPage,customer,domain,model,source,adspend,hardcosts,revenue,avg_polarity,...,links,percentSocial,percentDisplay,percentSearchPaid,percentSearch,visits,monthlyVisitsChange,bounceRate,dominantPlatform,global
0,kinobody.com,https://go.kinobody.com/quiz/,1,fitness,info / coaching,word of mouth,210000,20000,630000,0.074306,...,7,11.37,0.72,0.0391,49.97,335740,27229.0,62.19,Youtube,88504
1,quibi.com,https://go.kinobody.com/shred-bundle/,1,entertainment,software,youtube,9000,3200,150000,0.129296,...,18,6.51,0.00,0.0000,56.65,49999,39239.0,77.48,Youtube,477323


In [25]:
class Features:
    def __init__(self, df):
        self.df = df
        
    def pp(self):
        ps = self.df['percentSocial']
        pd = self.df['percentDisplay']
        psp = self.df['percentSearchPaid']
        n = len(ps)
        percent = []
        for i in range(n):
            paid = (ps[i]+pd[i]+psp[i])/100
            percent.append(paid)
        return percent

    def newCustomers(self, crt=0.01):
        visits = self.df['visits']
        bounce = self.df['bounceRate']
        customers = []
        for i in range(len(visits)):
            realVisits = visits[i] * (bounce[i]/100)
            customers.append(int(realVisits*crt))
        return customers
    
    def adRevenue(self):
        return self.df['revenue'] * self.df['revenue']
        '''revenue = self.df['revenue']
        pp = self.df['percentPaid']
        ar = []
        for i in range(len(revenue)):
            ar.append(revenue[i] * pp[i])
        return ar'''
    
    def cac(self):
        adspend = self.df['adspend']
        customers = self.df['newCustomers']
        cac = []
        for i in range(len(adspend)):
            cac.append(adspend[i]/customers[i])
        return cac
    
    def roas(self):
        adspend = self.df['adspend']
        ar = self.df['adRevenue']
        roas = []
        for i in range(len(adspend)):
            roas.append(ar[i]/adspend[i])
        return roas
    
    def profit(self):
        return self.df['revenue'] - self.df['adspend'] - self.df['hardcosts']
    
    def lpc(self):
        return self.df['words'] * self.df['triggers'] * self.df['links']
    
    def aov(self):
        return self.df['revenue'] / self.df['newCustomers']

In [26]:
def addFeats(df):
    f = Features(df)
    df['percentPaid'] = f.pp()
    df['newCustomers'] = f.newCustomers()
    return df

def aggregate(df):
    f = Features(df)
    df['adRevenue'] = f.adRevenue()
    df['cac'] = f.cac()
    df['roas'] = f.roas()
    df['profit'] = f.profit()
    df['profitMargin'] = df['profit']/df['revenue']
    df['lpc'] = f.lpc()
    df['averageOrderValue'] = f.aov()
    return df

df = db2df()
df2 = addFeats(df)
aggregate(df2)

,name,landingPage,customer,domain,model,source,adspend,hardcosts,revenue,avg_polarity,...,global,percentPaid,newCustomers,adRevenue,cac,roas,profit,profitMargin,lpc,averageOrderValue
0,kinobody.com,https://go.kinobody.com/quiz/,1,fitness,info / coaching,word of mouth,210000,20000,630000,0.074306,...,88504,0.121291,2087,76413.33,100.622904,0.363873,400000,0.634921,105833,301.868711
1,quibi.com,https://go.kinobody.com/shred-bundle/,1,entertainment,software,youtube,9000,3200,150000,0.129296,...,477323,0.065100,387,9765.00,23.255814,1.085000,137800,0.918667,2411676,387.596899
